# 📈 Análise Exploratória (EDA)
## 🔄 Ajuste do Diretório de Trabalho

Antes de carregar ou manipular arquivos, é importante garantir que estamos no diretório correto do projeto.  
O código abaixo verifica se o notebook está sendo executado a partir da pasta `notebooks`. Se for o caso, ele sobe um nível na hierarquia de diretórios para garantir que o diretório de trabalho seja a raiz do projeto.

Isso é útil para manter caminhos relativos consistentes ao acessar dados, scripts ou outros recursos do projeto.

📌 **Resumo do que o código faz:**
- Verifica se o diretório atual termina com `notebooks`.
- Se sim, volta uma pasta (para a raiz do projeto).
- Exibe o novo diretório de trabalho.


In [17]:
import os

# Verifica se o diretório de trabalho atual termina com 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    # Se sim, sobe um nível de diretório para a pasta raiz do projeto
    os.chdir('..')

# Imprime o diretório de trabalho para confirmar que a mudança foi feita
print(f"Diretório de Trabalho Atual: {os.getcwd()}")

Diretório de Trabalho Atual: c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov


## ⚙️ Carregamento de Bibliotecas e Configurações do Projeto

Nesta etapa, realizamos a importação das principais bibliotecas que serão utilizadas ao longo do projeto, organizadas por finalidade:

- 📁 **Configurações do Projeto**:  
  Importamos o módulo `src.config`, que armazena configurações como caminhos de arquivos, variáveis globais e parâmetros fixos.

- 📊 **Manipulação de Dados**:  
  - `pandas` e `numpy` para operações de estruturação e análise de dados.  
  - `duckdb` para consultas SQL diretamente em DataFrames.

- ⚙️ **Engenharia de Atributos**:  
  - `pandas_ta` é uma biblioteca especializada para análise técnica, útil para projetos com dados temporais (ex: séries financeiras).

- 📈 **Visualização de Dados**:  
  - `matplotlib.pyplot` e `seaborn` para construção de gráficos interativos e estilizados.

- 🧰 **Helpers do Notebook**:  
  - `display` do `IPython` facilita a visualização de objetos em células do notebook.

Além disso, aplicamos configurações visuais para padronizar os gráficos:
- Tema `whitegrid` do Seaborn.
- Estilo `fivethirtyeight` do Matplotlib.
- Habilitação do modo inline para que os gráficos sejam renderizados dentro do notebook.

```python
print("✅ Bibliotecas carregadas com sucesso!")


In [18]:
# --- Configurações do Projeto ---
# Importa nossas configurações (ex: caminho do DB)
import src.config as config

# --- Manipulação de Dados ---
import pandas as pd
import numpy as np
import duckdb

# --- Engenharia de Atributos ---
import pandas_ta as ta

# --- Visualização ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Helpers do Notebook ---
from IPython.display import display

# Garante que os gráficos sejam exibidos no notebook
%matplotlib inline 

print("✅ Bibliotecas carregadas com sucesso!")

✅ Bibliotecas carregadas com sucesso!


## 🧰 Configuração de Estilo e Exibição de Dados

Antes de iniciar a análise, definimos algumas configurações visuais e de exibição para melhorar a legibilidade dos outputs e a consistência gráfica.

### 📊 Configurações do `pandas`
- `max_columns = 100`: Permite visualizar até 100 colunas sem truncamento.
- `max_rows = 100`: Permite visualizar até 100 linhas diretamente no output.
- `float_format = '{:.2f}'`: Formata números decimais com 2 casas, facilitando a leitura.

```python
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
```

In [19]:
# Pandas display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Estilo visual com Seaborn (engloba Matplotlib)
sns.set_theme(
    context='notebook',
    style='whitegrid',
    palette='viridis',
    font='sans-serif',
    font_scale=1.1,
    rc={
        'figure.figsize': (12, 6),
        'font.size': 12,
        'axes.titlesize': 16,
        'axes.labelsize': 14,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'legend.fontsize': 12,
        'lines.linewidth': 2,
        'grid.alpha': 0.3
    }
)

## 🗃️ Carregamento dos Dados - Fase 1

Nesta etapa, conectamos ao banco de dados local gerado na **Fase 1** do pipeline para carregar os dados da tabela `precos_diarios`.

Utilizamos o **DuckDB**, um banco de dados analítico embutido, eficiente para análise local de dados com sintaxe SQL.

📌 **Etapas realizadas:**
1. O caminho do banco de dados (`DB_PATH`) é recuperado do módulo de configurações `config`.
2. Estabelecemos uma conexão com o DuckDB em modo **somente leitura** (`read_only=True`), garantindo integridade.
3. Executamos uma query SQL para obter todos os registros da tabela `precos_diarios`.
4. A conexão é encerrada após a leitura.
5. O dataframe resultante é exibido com:
   - `df.info()` para verificar a estrutura.
   - `df.head()` para visualizar as primeiras linhas.

Em caso de falha, uma mensagem de erro orienta o usuário a verificar se o pipeline da Fase 1 foi executado corretamente e se o arquivo do banco existe no local esperado.


In [20]:
# --- Carregando os dados da Fase 1 ---

# O caminho para o banco de dados já está no nosso módulo 'config'
db_path = str(config.DB_PATH)

print(f"Carregando dados de: {db_path}")

try:
    # Conecta ao banco de dados DuckDB em modo de apenas leitura
    con = duckdb.connect(database=db_path, read_only=True)

    # Executa uma query SQL para selecionar todos os dados da nossa tabela
    df = con.execute("SELECT * FROM precos_diarios").fetchdf()

    # Fecha a conexão com o banco de dados
    con.close()

    # --- Verificação ---
    print("\n✅ Dados carregados com sucesso! Verificando a estrutura:")
    df.info()

    print("\nExibindo as 5 primeiras linhas:")
    display(df.head())

except Exception as e:
    print(f"❌ Ocorreu um erro ao carregar os dados: {e}")
    print("Verifique se o pipeline da Fase 1 foi executado com sucesso e se o arquivo do banco de dados existe no local correto.")

Carregando dados de: C:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\data\mercados.duckdb

✅ Dados carregados com sucesso! Verificando a estrutura:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   data                  3000 non-null   object 
 1   close_petroleo_brent  3000 non-null   float64
 2   close_petrobras       3000 non-null   float64
 3   close_dolar           3000 non-null   float64
 4   close_ibovespa        3000 non-null   float64
 5   close_sp500           3000 non-null   float64
 6   high_petroleo_brent   3000 non-null   float64
 7   high_petrobras        3000 non-null   float64
 8   high_dolar            3000 non-null   float64
 9   high_ibovespa         3000 non-null   float64
 10  high_sp500            3000 non-null   float64
 11  low_petroleo_brent    3000 non-null   float64
 12  low_petrobras     

,data,close_petroleo_brent,close_petrobras,close_dolar,close_ibovespa,close_sp500,high_petroleo_brent,high_petrobras,high_dolar,high_ibovespa,high_sp500,low_petroleo_brent,low_petrobras,low_dolar,low_ibovespa,low_sp500,open_petroleo_brent,open_petrobras,open_dolar,open_ibovespa,open_sp500,semana_do_mes
0,2014-01-01,95.44,6.15,2.36,50341.00,1831.98,98.97,6.32,2.36,51656.00,1845.86,95.34,6.11,2.35,50246.00,1827.74,98.50,6.30,2.36,51522.00,1845.86,1
1,2014-01-02,95.44,6.15,2.36,50341.00,1831.98,98.97,6.32,2.41,51656.00,1845.86,95.34,6.11,2.35,50246.00,1827.74,98.50,6.30,2.36,51522.00,1845.86,1
2,2014-01-03,93.96,6.06,2.38,50981.00,1831.37,95.74,6.16,2.39,50981.00,1838.24,93.86,6.03,2.36,50269.00,1829.13,95.47,6.15,2.38,50348.00,1833.21,1
3,2014-01-06,93.43,6.10,2.37,50974.00,1826.77,94.59,6.11,2.38,51002.00,1837.16,93.20,5.93,2.36,50451.00,1823.73,94.18,6.04,2.37,50980.00,1832.31,1
4,2014-01-07,93.67,5.93,2.37,50430.00,1837.88,94.22,6.18,2.37,51478.00,1840.10,93.35,5.92,2.34,50429.00,1828.71,93.60,6.11,2.37,50982.00,1828.71,1


## 🗓️ Ajuste do Índice Temporal

Para garantir uma manipulação adequada de séries temporais, realizamos o ajuste da coluna `data`, transformando-a no índice principal do DataFrame.

📌 **Etapas executadas:**
1. Convertida a coluna `'data'` para o tipo `datetime` com `pd.to_datetime()` para garantir que operações temporais funcionem corretamente (ex: resampling, filtros por período).
2. Definida como **índice do DataFrame** com `df.set_index('data', inplace=True)`, substituindo o índice numérico padrão.
3. Exibimos as primeiras linhas para confirmar a alteração, verificando que `'data'` agora aparece como índice à esquerda do DataFrame.

Esse passo é essencial em análises temporais, especialmente para séries de preços, permitindo ordenação cronológica, reamostragem e outras operações baseadas em tempo.


In [21]:
# Garante que a coluna 'data' esteja no formato datetime, caso ainda não esteja
df['data'] = pd.to_datetime(df['data'])

# Define a coluna 'data' como o novo índice do DataFrame
# O inplace=True modifica o DataFrame diretamente
df.set_index('data', inplace=True)

# Exibe as primeiras linhas para confirmar a mudança. 
# Note que 'data' agora está no lugar do índice numérico.
print("✅ Coluna 'data' definida como índice. A estrutura agora é:")
display(df.head())

✅ Coluna 'data' definida como índice. A estrutura agora é:


,close_petroleo_brent,close_petrobras,close_dolar,close_ibovespa,close_sp500,high_petroleo_brent,high_petrobras,high_dolar,high_ibovespa,high_sp500,low_petroleo_brent,low_petrobras,low_dolar,low_ibovespa,low_sp500,open_petroleo_brent,open_petrobras,open_dolar,open_ibovespa,open_sp500,semana_do_mes
data,,,,,,,,,,,,,,,,,,,,,
2014-01-01,95.44,6.15,2.36,50341.00,1831.98,98.97,6.32,2.36,51656.00,1845.86,95.34,6.11,2.35,50246.00,1827.74,98.50,6.30,2.36,51522.00,1845.86,1
2014-01-02,95.44,6.15,2.36,50341.00,1831.98,98.97,6.32,2.41,51656.00,1845.86,95.34,6.11,2.35,50246.00,1827.74,98.50,6.30,2.36,51522.00,1845.86,1
2014-01-03,93.96,6.06,2.38,50981.00,1831.37,95.74,6.16,2.39,50981.00,1838.24,93.86,6.03,2.36,50269.00,1829.13,95.47,6.15,2.38,50348.00,1833.21,1
2014-01-06,93.43,6.10,2.37,50974.00,1826.77,94.59,6.11,2.38,51002.00,1837.16,93.20,5.93,2.36,50451.00,1823.73,94.18,6.04,2.37,50980.00,1832.31,1
2014-01-07,93.67,5.93,2.37,50430.00,1837.88,94.22,6.18,2.37,51478.00,1840.10,93.35,5.92,2.34,50429.00,1828.71,93.60,6.11,2.37,50982.00,1828.71,1
